<a href="https://colab.research.google.com/github/SyaoranClone/Persona-Chatbot/blob/master/Persona_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec 16 07:15:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#install Apex
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
import json
import time
import os
import random
import warnings
from itertools import chain
from argparse import ArgumentParser
from collections import defaultdict
from functools import partial
from tqdm import tqdm, trange
from tqdm import tnrange, notebook

import torch
import numpy as np
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, DistributedSampler, SequentialSampler
from transformers import (AdamW, OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer,
                                 GPT2DoubleHeadsModel, GPT2Tokenizer, WEIGHTS_NAME, CONFIG_NAME)
from transformers import get_linear_schedule_with_warmup

In [ ]:
!pip install spacy ftfy==4.4.3

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for ftfy: filename=ftfy-4.4.3-cp36-none-any.whl size=41069 sha256=4fd874edf9ef42542fc6e354d3fb61ec22896f2c7dcff6784ccaeb3976c761c0
  Stored in directory: /root/.cache/pip/wheels/37/54/00/d320239bfc8aad1455314f302dd82a75253fc585e17b81704e
Successfully built ftfy


In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
torch.cuda.device_count()

1

In [ ]:
#According to Huggingface Convai tutorial
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>']}
MODEL_INPUTS = ["input_ids", "mc_token_ids", "lm_labels", "mc_labels", "token_type_ids"]
PADDED_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]

In [ ]:
#define hyperparameters
class args:
  model_checkpoint = 'openai-gpt'
  device  = 'cuda' if torch.cuda.is_available() else 'cpu'
  lr  = 6.25e-5
  num_candidates = 4
  personality_permutations = 2
  num_history  = 2 #Number of previous exchanges to keep in history
  fp16_training = "O1" #Set to O0, O1, O2 or O3 for fp16 training
  train_batch_size = 2
  valid_batch_size = 2
  num_epochs= 2
  no_sample=False
  lm_coef = 2.0
  mc_coef = 1.0
  max_norm= 1.0
  top_p = 0.9 # Nucleus filtering (top-p) before sampling (<=0.0: no filtering)
  top_k = 0 # Filter top-k tokens before sampling (<=0: no filtering)
  temperature = 0.7 # Sampling softmax temperature
  max_len = 20 #Maximum length of the output utterances
  min_len = 1
  num_gpu = torch.cuda.device_count() #1
  gradient_accumulation_steps= 4
  local_rank= -1 # for distributed training
  url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"
  local_path= "/content/drive/My Drive/Colab Notebooks/Dataset/personachat_self_original.json"
  dataset_cache_path = "/content/drive/My Drive/Colab Notebooks/Dataset/dataset_cache/persona_cache.bin_GPTTokenizer"
  saved_dir = "/content/drive/My Drive/Colab Notebooks/Trained Models/persona_chatbot/"
  longest_common = 5
args = args

In [ ]:
class FacebookPersonaDataset():
  """
    Concatenate context segments in a single sequence [[bos+persona], [history], [reply+eos]]
    Tokenize and convert them to tensor
    
  """
  def __init__(self,url,cache_path,tokenizer = ''):
    self.file_path = url
    self.tokenizer = tokenizer
    self.cache_path = cache_path

  def load_dataset(self):
    #self.cache_path = self.cache_path + '_' + type(self.tokenizer).__name__  # To avoid using GPT cache for GPT-2 and vice-versa
    if self.cache_path and os.path.isfile(self.cache_path):
      #load tokenized dataset
      dataset = torch.load(self.cache_path)
      print("dataset loaded")
    else:
      with open(self.file_path, "r", encoding="utf-8") as f:
        dataset = json.loads(f.read())
      
      def tokenize(obj):
        if isinstance(obj,str):
          return self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(obj))
        if isinstance(obj,dict):
          return dict((n, tokenize(o)) for n, o in obj.items())
        return list(tokenize(o) for o in obj)
      dataset = tokenize(dataset)
      torch.save(dataset,self.cache_path)
    return dataset

  def _pad_dataset(self,dataset, padding=0):
    """ Pad the dataset. This could be optimized by defining a Dataset class and padding at the batch level, but this is simpler. """
    max_l = max(len(x) for x in dataset["input_ids"])
    for name in PADDED_INPUTS:
        dataset[name] = [x + [padding if name != "lm_labels" else -100] * (max_l - len(x)) for x in dataset[name]]
    return dataset

  def build_input(self,persona,history,reply,lm_labels = False,with_eos = True):
    """ Build a sequence of input from 3 segments: persona, history and last reply. """
    bos, eos, speaker1, speaker2 = self.tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    #bos, eos, speaker1, speaker2 = "<bos>", "<eos>", "<speaker1>", "<speaker2>"
    #sequence = [[bos] + list(persona) + ["his"] + history + ["rep"] + [reply]]
    sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])] #add bos, eos
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])] #add speaker1, speaker2
    #after concat: [[bos+persona], [history], [reply+eos]]
    instance = {}
    instance["input_ids"] = list(chain(*sequence))
    #segment
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    #position
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    #language model labels is used to calculate lm_loss
    instance["lm_labels"] = [-100] * len(instance["input_ids"]) #labels set to -100 are ignored (masked)
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance

  def get_data_loaders(self):
    personachat_dataset = self.load_dataset()
    datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    for name,dataset in personachat_dataset.items():
      num_candidates = len(dataset[0]["utterances"][0]["candidates"]) #num_candidates are same for all dialoges
      if args.num_candidates > 0 and name == 'train':
        num_candidates =  min(num_candidates ,args.num_candidates)
      for dialoge in dataset:
        persona = dialoge["personality"].copy()
        for _ in range(args.personality_permutations):
          for utterance in dialoge["utterances"]:
            history = utterance["history"][-(2*args.num_history+1):]
            for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
              #The last candidate is a ground truth reponse.
              lm_labels = bool(j==num_candidates-1)
              instance = self.build_input(persona,history,candidate,lm_labels)
              for input_name,data in instance.items():
                datasets[name][input_name].append(data) #datasets['train']['input_ids'] of [[c1 in u1],[c2 in u1],..,[c1 in u 7][c2 in u7]]
            datasets[name]["mc_labels"].append(num_candidates - 1) #7
            datasets[name]["n_candidates"] = num_candidates
          persona = [persona[-1]] + persona[:-1]  # permuted personalitie
          
    #pad input and convert to tensor
    print("pad input and convert to tensor")
    tensor_datasets = {"train": [], "valid": []}
    for dataset_name, dataset in datasets.items():
      dataset =  self._pad_dataset(dataset, padding=self.tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
      for input_name in MODEL_INPUTS:
        tensor =  torch.tensor(dataset[input_name])
        if input_name != "mc_labels":
          tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
        tensor_datasets[dataset_name].append(tensor)

    train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
    return train_dataset, valid_dataset 

In [ ]:
def add_special_tokens(model,tokenizer):
  origin_num_tokens = len(tokenizer.encoder)
  num_special_tokens = tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
  if num_special_tokens > 0:
        model.resize_token_embeddings(new_num_tokens=origin_num_tokens + num_special_tokens)

In [ ]:
model_class_name = GPT2DoubleHeadsModel if "gpt2" in args.model_checkpoint else OpenAIGPTDoubleHeadsModel
tokenizer_class_name = GPT2Tokenizer if "gpt2" in args.model_checkpoint else OpenAIGPTTokenizer
model = model_class_name.from_pretrained(args.model_checkpoint)
tokenizer = tokenizer_class_name.from_pretrained(args.model_checkpoint)
model.to(args.device)

In [ ]:
add_special_tokens(model,tokenizer)

In [ ]:
def train(train_dataset,valid_dataset,model,tokenizer):
  train_sampler = RandomSampler(train_dataset) if args.local_rank== -1 else DistributedSampler(train_dataset)
  train_loader = DataLoader(train_dataset,batch_size=args.train_batch_size,sampler=train_sampler)
  optimizer = AdamW(model.parameters(),lr = args.lr,correct_bias=True)

  if args.fp16_training:
    from apex import amp
    # Allow Amp to perform casts as required by the opt_level 
    model,optimizer = amp.initialize(model,optimizer,opt_level=args.fp16_training)
  #Linearly decrease the learning rate from lr to zero
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = args.num_epochs* len(train_loader))
  tr_loss, loss, last_saved_loss = 0.0, 0.0, 2.0
  global_step = 0
  metrics = {
      "nll" : 1000.0,
      "accuracy": 0.0
  }
  model.zero_grad() # Reset gradient tensor
  if args.num_gpu > 1:
    model = torch.nn.DataParallel(model)

  if args.local_rank != -1:
    model = DistributedDataParallel(model,device_ids = [args.local_rank],output_device=args.local_rank,find_unused_parameters=True)
  
  #eval model when we start the training and at the end of each epoch
  if args.local_rank in [-1,0]:
    metrics = evaluate(model, valid_dataset, metrics, tokenizer)
  for _ in range(args.num_epochs):
    for step,batch in enumerate(notebook.tqdm(train_loader,disable= args.local_rank not in [-1,0])):
      model.train()
      batch = tuple(input_tensor.to(args.device) for input_tensor in batch )
      input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch

      (lm_loss),(mc_loss), *_ = model(input_ids,token_type_ids=token_type_ids, mc_token_ids=mc_token_ids,mc_labels=mc_labels,lm_labels=lm_labels)

      loss = (lm_loss * args.lm_coef + mc_loss * args.mc_coef)/args.gradient_accumulation_steps # Normalize our loss (if averaged)

      if args.num_gpu > 1:
        # mean() to average on multi-gpu parallel training
        loss = loss.mean()

      if step % 1000 == 0:
        print("Loss for step {} is {}".format(step, loss))
        #Save model
        if last_saved_loss > loss:
          model_to_save = model.module if hasattr(model, "module") else model #Take care of distributed/parallel training
          model_to_save.save_pretrained(args.saved_dir)
          tokenizer.save_pretrained(args.saved_dir)
          last_saved_loss = loss


      if args.fp16_training:
        with amp.scale_loss(loss, optimizer) as scaled_loss:
          scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_norm)
      else:
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0. This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(),args.max_norm)

      tr_loss += loss.item()
      global_step += 1
      if (step+1) % args.gradient_accumulation_steps == 0: # Wait a several backward step
        optimizer.step()
        model.zero_grad()
        # if args.local_rank in [-1,0]:
        #   metrics = evaluate(model, valid_dataset, metrics, tokenizer)
      # Update the learning rate.
      scheduler.step()
    #eval model 
    if args.local_rank in [-1,0]:
      metrics = evaluate(model, valid_dataset, metrics, tokenizer)
        
  return tr_loss/global_step,metrics


In [ ]:
def evaluate(model,valid_dataset,metrics,tokenizer):
  valid_sample = SequentialSampler(valid_dataset) if  args.local_rank== -1 else DistributedSampler(train_dataset)
  eval_dataloader = DataLoader(valid_dataset,sampler=valid_sample,batch_size=args.valid_batch_size)

  print(' * Running Evaluation')
  print(' * Num of examples: %d" ',len(valid_dataset))
  print(' * Batch size: %d" ',args.valid_batch_size)
  nlls = None
  accs = None
  for step,batch in enumerate(notebook.tqdm(eval_dataloader,desc="Evaluating")):
    model.eval()
    with torch.no_grad():
      batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
      input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch
      lm_logits, mc_logits, *_ = model(
          input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids)
      
      lm_logits_flat_shifted = lm_logits[..., :-1, :].contiguous().view(-1, lm_logits.size(-1))
      lm_labels_flat_shifted = lm_labels[..., 1:].contiguous().view(-1)
            
      x = ((lm_logits_flat_shifted, mc_logits), (lm_labels_flat_shifted, mc_labels))
      #convert pytorch tensor to numpy array to calculate
      nll = torch.nn.CrossEntropyLoss(ignore_index=-100)(x[0][0], x[1][0]).detach().cpu().numpy()
      acc = torch.sum((torch.max(x[0][1], 1)[1] == x[1][1]).int()).detach().cpu().numpy().mean()

    if nlls is None:
      nlls = nll
      accs = acc
    else:
      nlls = np.append(nlls, nll)
      accs = np.append(accs, acc)
  
  nlls_mean = np.mean(nlls)
  accs_mean = np.mean(accs)
    
  if accs_mean>metrics['accuracy'] and nlls_mean<metrics['nll']:
    print(" * New high accuracy and nll! {} {} ".format(accs_mean, nlls_mean))
    metrics.update({'nll': nlls_mean, 'accuracy': accs_mean})
    #Save model if getting high accuracy
    model_to_save = model.module if hasattr(model, "module") else model #Take care of distributed/parallel training
    model_to_save.save_pretrained(args.saved_dir)
    tokenizer.save_pretrained(args.saved_dir)
    
  return metrics 

In [ ]:
persona_dataset = FacebookPersonaDataset(args.local_path,args.dataset_cache_path,tokenizer)

In [ ]:
train_dataset, valid_dataset = persona_dataset.get_data_loaders() 

dataset loaded
pad input and convert to tensor


In [ ]:
train_dataset.tensors[0].shape

torch.Size([262876, 4, 282])

In [ ]:
total_loss, metric = train(train_dataset,valid_dataset,model,tokenizer)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
 * Running Evaluation
 * Num of examples: %d"  15602
 * Batch size: %d"  2



 * New high accuracy and nll! 0.0917831047301628 5.24331521987915 


/usr/local/lib/python3.6/dist-packages/transformers/modeling_openai.py:689: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:127: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Loss for step 0 is 2.730668306350708
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Loss for step 1000 is 2.8442227840423584
Loss for step 2000 is 1.5790939331054688
Loss for step 3000 is 2.2540135383605957
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Loss for step 4000 is 1.1778292655944824
Loss for step 5000 is 1.6517231464385986
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Loss for step 6000 is 1.6394068002700806
Loss for step 7000 is 1.6226752996444702
Loss for step 8000 is 1.0814038515090942
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Loss for step 9000 is 1.7652204036712646
Loss for step 10000 is 1.3035157918930054
Loss for step 11000 is 1.8181471824645996
Loss for step 12000 is 1.60902523994

In [ ]:
#Interact
def top_filtering(logits, top_k = 0, top_p = 0.9,threshold = -float('Inf'),filter_value=-float('Inf')):
  """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
            threshold: a minimal threshold to keep logits
  """
  assert logits.dim() == 1 #batch_size = 1
  top_k = min(top_k,logits.size(-1))
  if top_k > 0:
    # Remove all tokens with a probability less than the last token in the top-k tokens
    indices_to_remove = logits < torch.top_k(logits,top_k)[0][...,-1,None]
    logits[indices_to_remove] = filter_value
  
  if top_p > 0.0:
    # Compute cumulative probabilities of sorted tokens
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    # Remove tokens with cumulative probability above the threshold
    sorted_indices_to_remove = cumulative_probabilities > top_p
    # Shift the indices to the right to keep also the first token above the threshold
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    # Back to unsorted indices and set them to -infinity
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value
  
  indices_to_remove = logits < threshold
  logits[indices_to_remove] = filter_value

  return logits

In [ ]:
def sample_sequence(personality,history,tokenizer,model,persona_dataset,current_output = None):
  """
    Generate reponse from previous reponses
  """
  special_tokens_ids = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
  if current_output is None:
    current_output = []
  
  for i in range (args.max_len):
    instance = persona_dataset.build_input(personality,history,current_output,with_eos=False)
    input_ids = torch.tensor(instance["input_ids"],device= args.device).unsqueeze(0)
    token_type_ids =  torch.tensor(instance["token_type_ids"], device= args.device).unsqueeze(0)
    
    logits = model(input_ids,token_type_ids = token_type_ids)
    if isinstance(logits, tuple):  # for gpt2 and maybe others
      logits = logits[0]
    logits = logits[0] 
    # logits shape (batch_size, num_choices, sequence_length, vocab_size)
    logits = logits[0,-1,:]/args.temperature 
    logits = top_filtering(logits,top_k = args.top_k,top_p = args.top_p)
    probs = F.softmax(logits, -1)

    prev = torch.topk(probs,1)[1] if args.no_sample else torch.multinomial(probs,1)
    
    if i < args.min_len and prev.item() in special_tokens_ids:
      while prev.item() in special_tokens_ids:
        if probs.max().item() == 1:
          warnings.warn("Warning: model generating special token with probability 1.")
          break  # avoid infinitely looping over special token
        prev = torch.multinomial(probs, num_samples=1)
    
    if prev.item() in special_tokens_ids:
      break
    current_output.append(prev.item())
      
  return current_output

In [ ]:
dataset = persona_dataset.load_dataset()
personalities = [dialog["personality"] for dataset in dataset.values() for dialog in dataset]
personality = random.choice(personalities)

dataset loaded


In [ ]:
def predict(model,tokenizer,personality,persona_dataset):
  print("Selected personality: %s", tokenizer.decode(chain(*personality)))
  history = []
  out_ids = []
  while True:
    raw_text = input(">>> ")
    while not raw_text:
      print('Prompt should not be empty!')
      raw_text = input(">>> ")
    history.append(tokenizer.encode(raw_text))
    with torch.no_grad():
      #out_ids = sample_sequence(personality, history, tokenizer, model, persona_dataset) 
      while True:
        out_ids = sample_sequence(personality, history, tokenizer, model, persona_dataset) 
        print("out_ids: ", out_ids)
        if not check_repetition_cross_turn(history,out_ids):
          break
        else:
          print("long common: ", tokenizer.decode(out_ids, skip_special_tokens=True))
          if not out_ids: # avoid infinitely looping over special token
            break
    history.append(out_ids)
    history = history[-(2*args.num_history+1):]
    out_text = tokenizer.decode(out_ids, skip_special_tokens=True)
    print(out_text)

In [ ]:
def check_repetition_cross_turn(history,output):
  if not output:
    return True
  for seq in history:
    if lcs(seq,output) >= args.longest_common:
      return True
  return False

In [ ]:
def lcs(X, Y): 
    # find the length of the strings 
    m = len(X) 
    n = len(Y) 
  
    # declaring the array for storing the dp values 
    L = [[None]*(n + 1) for i in range(m + 1)] 
    # print(L)
  
    """Following steps build L[m + 1][n + 1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of X[0..i-1] 
    and Y[0..j-1]"""
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1] 
    return L[m][n] 

In [ ]:
#reload checkpoints and evaluate on test dataset
model_class_name = GPT2DoubleHeadsModel if "gpt2" in args.model_checkpoint else OpenAIGPTDoubleHeadsModel
tokenizer_class_name = GPT2Tokenizer if "gpt2" in args.model_checkpoint else OpenAIGPTTokenizer
model = model_class_name.from_pretrained(args.saved_dir)
tokenizer =  tokenizer_class_name.from_pretrained(args.saved_dir) #, do_lower_case=True)
model.to(args.device)

In [ ]:
#test
lines = open("/content/drive/My Drive/Colab Notebooks/Dataset/personality.txt").read().split('\n')[:-1]
lines = [line.lower() for line in lines]

In [ ]:
raw_personality = ["my name is lisa .","i like to remodel homes .", "i like to go hunting .","i like to shoot a bow .", "my favorite holiday is halloween ."]

In [ ]:
#convert to ids
def tokenize(obj):
  if isinstance(obj,str):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
  if isinstance(obj,dict):
    return dict((n, tokenize(o)) for n, o in obj.items())
  return list(tokenize(o) for o in obj)
personality = tokenize(raw_personality)

In [ ]:
personality

[[547, 1362, 544, 6544, 239],
 [249, 649, 485, 5371, 4621, 7654, 239],
 [249, 649, 485, 799, 5408, 239],
 [249, 649, 485, 4008, 246, 2877, 239],
 [547, 3898, 6974, 544, 13537, 239]]

In [ ]:
tokenizer.decode(chain(*personality))

'my name is lisa. i like to remodel homes. i like to go hunting. i like to shoot a bow. my favorite holiday is halloween.'

In [ ]:
personality

[[72, 588, 284, 711, 9283, 764],
 [1820, 4004, 1295, 318, 262, 12269, 764],
 [72, 423, 362, 3988, 764],
 [72, 716, 257, 6253, 764]]

In [ ]:
personalities = [dialog["personality"] for dataset in dataset.values() for dialog in dataset]
personality = random.choice(personalities)

In [ ]:
predict(model,tokenizer,personality,persona_dataset)

Selected personality: %s my name is lisa. i like to remodel homes. i like to go hunting. i like to shoot a bow. my favorite holiday is halloween.
>>> hi
out_ids:  [3569, 655, 240, 718, 640, 512, 257]
hi there, how are you?
>>> hi
out_ids:  [3569, 267, 249, 1048, 862, 2190, 239, 718, 670, 512, 257]
hi! i am well thanks. how about you?
>>> hi
out_ids:  [3569, 240, 249, 719, 3533, 240, 488, 249, 719, 246, 6070, 3300, 239]
hi, i'm anne, and i'm a professional hunter.
>>> hi
out_ids:  [249, 719, 6544, 488, 249, 1119, 485, 4008, 8493]
i'm lisa and i love to shoot deer
>>> hi
out_ids:  [249, 719, 6544, 240, 488, 249, 1119, 485, 5371, 4621, 7654, 239]
long common:  i'm lisa, and i love to remodel homes.
out_ids:  [249, 719, 4971]
i'm bob
>>> hi
out_ids:  [599, 587, 512, 587, 562, 246, 1885, 257]
what do you do for a living?
>>> hi
out_ids:  [249, 1048, 246, 3337, 4721, 2351]
i am a computer programmer
>>> hi
out_ids:  [249, 719, 6544]
i'm lisa
>>> i'm lisa
out_ids:  [599, 587, 512, 587, 562, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Warning: model generating special token with probability 1.


out_ids:  []
long common:  

>>> okay
out_ids:  [599, 587, 512, 587, 562, 246, 1885]
what do you do for a living


KeyboardInterrupt: ignored

In [ ]:

x = (([1,2,3],[[3,4,3],[5,8,5],[6,6,11]]),([3,4,3],[5,5,5]))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.sum((torch.max(torch.tensor(x[0][1]), 1)[1] == torch.tensor(x[1][1])).int()).detach().cpu().numpy().mean()

0.0

In [ ]:
X = [1820, 1438, 318, 312, 313, 764]
Y = [1820, 1438, 318, 300, 9160, 764]
print("Length of LCS is ", lcs(X, Y))

[[None, None, None, None, None, None, None], [None, None, None, None, None, None, None], [None, None, None, None, None, None, None], [None, None, None, None, None, None, None], [None, None, None, None, None, None, None], [None, None, None, None, None, None, None], [None, None, None, None, None, None, None]]
Length of LCS is  4
